In [1]:
import numpy as np
import random

random.seed(100)

def compute_pi(num_samples, num_bins = 20):
    results = []
    counter = 0

    for ib in range(num_bins):
        counter = 0
        for i in range(num_samples):
             x = 2 * random.random() - 1.0
             y = 2 * random.random() - 1.0
             r = x**2 + y**2
             if r <= 1.0:
                counter += 1
        results.append(((4.*counter)/num_samples))
   
    return np.average(results), np.std(results)

for num_samples in [10**2, 10**3, 10**4, 10**5, 10**6]:
    av,err = compute_pi(num_samples)
    print(av, err, np.abs(av-np.pi))
    
    
#plt.errorbar(num_samples_list, data[:,0], data[:,1])
plt.plot(num_samples_list, data[:,1], marker="o")
plt.xscale("log")
plt.yscale("log")
plt.plot(num_samples_list, 1/np.array(num_samples_list, dtype=float)**0.5)

plt.xlabel("Number of samples")
plt.ylabel("Error bars")
plt.tight_layout()
plt.savefig("error.pdf")
    

3.142 0.110616454472 0.000407346410207
3.1392 0.0431666537967 0.00239265358979
3.14898 0.0196653909191 0.00738734641021
3.140668 0.00458125048431 0.000924653589793
3.1416988 0.00173655111068 0.000106146410207


array([ 0.,  1.,  2.])

In [20]:
printA()

A


In [17]:
import numpy as np
import random
#from abc import abstractmethod

class MonteCarloSampler:
    def __init__(self):
        self.results = []
    
    def perform(self):
        #do nothing
        print("Should not be called")
        
    def get_results(self):
        if len(self.results) == 0:
            print("No available results found")
            return 0.0
        return np.average(self.results)
    
    def reset(self):
        self.results = []

class SimpleMonteCarloSampler(MonteCarloSampler):
    def __init__(self):
        super().__init__()
        
    def perform(self):
        counter = 0
        num_samples = 10**3
        for i in range(num_samples):
            x = 2 * random.random() - 1
            y = 2 * random.random() - 1
            if x**2 + y**2 < 1:
                counter += 1
        self.results.append(4*counter/num_samples)

#class FastMonteCarloSampler(MonteCarloSampler):
#    def perform(self):
        #"""""******"""

mc = SimpleMonteCarloSampler()
mc.perform()

for i in range(10):
    mc.perform()
    print(mc.get_results())

mc.reset()
mc.perform()
print(mc.get_results())

3.124
3.104
3.079
3.0872
3.09733333333
3.11142857143
3.118
3.11733333333
3.1204
3.12436363636
3.28


In [17]:
import numpy as np
import random

class MonteCarloSampler:
    def __init__(self, params):
        self.results = []
    
    def perform(self):
        #do nothing
        print("Should not be called")
        
    def get_results(self):
        if len(self.results) == 0:
            print("No available results found")
            return 0.0
        return np.average(self.results)
    
    def reset(self):
        self.results = []

p = {"h" : 0.1, "Ns" : 1, "beta" : 10.0}
print(p["Ns"])
p["J"] = -1.0
p["Ns"] = 2

a = np.identity(2, dtype=float)
p["array"] = np.array(a)
p["array"] *= 2
print(a)
print(p)
mc = MonteCarloSampler(p)
#mc.perform()

1
[[ 1.  0.]
 [ 0.  1.]]
{'array': array([[ 2.,  0.],
       [ 0.,  2.]]), 'h': 0.1, 'Ns': 2, 'J': -1.0, 'beta': 10.0}
